In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from utils import *

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
model, tokenizer = get_classification_bert_model("gchhablani/bert-base-cased-finetuned-qnli")
model.load_state_dict(torch.load("/scr/models/LC/models_archive/Bert/base-ft-qnli/orig_qnli.pth"))
model_LC = get_LC_model_bert(model, num_GP_layers=1)

Setting up gated layer bert.encoder.layer.11


In [30]:
train_dataloader, validation_dataloader, dataset = get_glue_task_dataset('qnli', tokenizer)

In [31]:
standard_evaluate(model, dataset["validation"], "qnli")

/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.2749437689781189,
 'eval_accuracy': 0.9099395936298736,
 'eval_runtime': 20.7598,
 'eval_samples_per_second': 263.153,
 'eval_steps_per_second': 8.237}

In [5]:
get_layer_gates_loss(model_LC)

Layer bert.encoder.layer.11: loss:  tensor([0.9801], grad_fn=<PowBackward0>)


In [9]:
train_with_trainer(model_LC, dataset["train"], dataset["validation"], epochs=0.3,
                    eval_batch_size=8, train_batch_size=8, gp_weight=5, learning_rate=0.0001, task="sst2", lr_decay=0.99, weight_decay=0, use_sgd=False)

Step,Training Loss
10,29.104600
20,28.816900
30,28.064900
40,27.630200
50,27.105100
60,26.494200
70,26.105200
80,25.368400
90,25.074300
100,24.471100


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/soheil/miniconda3/envs/GP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [10]:
get_layer_gates_loss(model_LC)

Layer bert.encoder.layer.11: loss:  tensor([0.0943], device='cuda:0', grad_fn=<PowBackward0>)


In [7]:
collapse_model(model_LC)

Collapsing layer bert.encoder.layer.11


In [11]:
standard_evaluate(model_LC, dataset["validation"], "sst2")

{'eval_loss': 0.36678770184516907,
 'eval_accuracy': 0.8864678899082569,
 'eval_runtime': 3.3662,
 'eval_samples_per_second': 259.045,
 'eval_steps_per_second': 8.318}

In [22]:
get_num_parameters(model_LC)

105351938

In [23]:
get_num_parameters(model)

109483778

In [24]:
d = model_LC.state_dict()

In [28]:
for k in d.keys():
    if "act" in k:
        print(k)

In [27]:
# remove key "bert.encoder.layer.11.intermediate.act.weight" from model_LC.state_dict()
d.pop("bert.encoder.layer.11.intermediate.act.weight")

tensor([0.6930], device='cuda:0')